In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn

In [2]:
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

In [3]:
print(train_data.shape)
print(test_data.shape)

(47439, 41)
(31626, 40)


In [4]:
all_features = pd.concat((train_data.iloc[:,1:-1], test_data.iloc[:,1:]))

In [5]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / x.std()
)
all_features[numeric_features] = all_features[numeric_features].fillna(0)
input_features = all_features[numeric_features]

In [6]:
num_train = train_data.shape[0]

train_features = torch.tensor(input_features[:num_train].values, dtype=torch.float32)
train_labels = torch.tensor(train_data['Sold Price'].values.reshape(-1,1), dtype=torch.float32)

test_features = torch.tensor(input_features[num_train:].values, dtype=torch.float32)

In [7]:
train_dataset = torch.utils.data.TensorDataset(train_features,train_labels)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128)

In [8]:
in_features = train_features.shape[1]
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = nn.Sequential(
    nn.Linear(in_features,128),
    nn.Linear(128, 64),
    nn.Linear(64, 1)
)
net.to(device)

Sequential(
  (0): Linear(in_features=19, out_features=128, bias=True)
  (1): Linear(in_features=128, out_features=64, bias=True)
  (2): Linear(in_features=64, out_features=1, bias=True)
)

In [9]:
def log_rmse(net, features, labels):
    # 为了在取对数时进一步稳定该值，将小于1的值设置为1
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds),
                           torch.log(labels)))
    return rmse.item()

In [10]:
loss = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)

In [11]:
def train(dataloader, net, loss, optimizer):
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        y_hat = net(X)
        l = loss(y_hat, y)
        l.backward()
        optimizer.step()
        if batch % 50 == 0:
            running_loss = log_rmse(net, X, y)
            current = batch * len(X)
            print('Train Loss: %.4f, [%d//%d]' % (running_loss, current, len(dataloader.dataset)))


In [12]:
print('training on', device)
epochs = 10
for epoch in range(epochs):
    print('epoch:', epoch+1)
    train(train_dataloader, net, loss, optimizer)
print('Done!')

training on cpu
epoch: 1
Train Loss: 13.7064, [0//47439]
Train Loss: 10.0043, [6400//47439]
Train Loss: 8.6673, [12800//47439]
Train Loss: 9.4496, [19200//47439]
Train Loss: 7.8336, [25600//47439]
Train Loss: 8.2774, [32000//47439]
Train Loss: 8.0186, [38400//47439]
Train Loss: 8.6179, [44800//47439]
epoch: 2
Train Loss: 6.7405, [0//47439]
Train Loss: 6.9012, [6400//47439]
Train Loss: 5.7856, [12800//47439]
Train Loss: 5.8757, [19200//47439]
Train Loss: 4.4208, [25600//47439]
Train Loss: 3.9636, [32000//47439]
Train Loss: 3.7307, [38400//47439]
Train Loss: 4.5824, [44800//47439]
epoch: 3
Train Loss: 0.7376, [0//47439]
Train Loss: 2.3053, [6400//47439]
Train Loss: 2.7251, [12800//47439]
Train Loss: 1.7811, [19200//47439]
Train Loss: 2.2698, [25600//47439]
Train Loss: 1.2437, [32000//47439]
Train Loss: 1.5297, [38400//47439]
Train Loss: 1.2546, [44800//47439]
epoch: 4
Train Loss: 0.4145, [0//47439]
Train Loss: 1.5783, [6400//47439]
Train Loss: 1.2000, [12800//47439]
Train Loss: 0.3228, [